In [1]:
from multimodalrec.multimodalrec import MultimodalRec

Using TensorFlow backend.


In [2]:
from multimodalrec.multimodalrec import data_pipeline
from multimodalrec.model import model

In [3]:
from multimodalrec.lossTypes import RMSELossGraph

In [4]:
import numpy as np
import pandas as pd

In [5]:
import tensorflow as tf

In [6]:
recmodel = MultimodalRec()

In [7]:
recmodel.organize_multimodal_data(load=True)

Training User-Movie Latent Factors are extracting...
The sparsity level of training dataset is 95.1%


  6%|▋         | 188/2930 [00:00<00:01, 1874.09it/s]

Done.
Visual Representations are extracting...


100%|██████████| 2930/2930 [00:01<00:00, 2191.78it/s]

Done.


In [8]:
# self.user_item_network_training.CF_data, self.user_latent_traninig, 
# self.visual_features

In [9]:
len(recmodel.user_latent_traninig.keys())

6040

In [10]:
len(recmodel.visual_features.keys())

2930

In [11]:
ratings_df_training = recmodel.user_item_network_training.CF_data

In [12]:
# TRAIN
# Create X_train y_train (POSITIVE)
pos_ratings_df = ratings_df_training[ratings_df_training.Rating>4]
pos_ratings_df = pos_ratings_df.assign(Likes= lambda x: 1)

# Create X_train y_train (NEGATIVE)
neg_ratings_df = ratings_df_training[ratings_df_training.Rating<3]
neg_ratings_df = neg_ratings_df.assign(Likes= lambda x: -1)

training_df = pd.concat([pos_ratings_df,neg_ratings_df],axis=0)
training_df = training_df.drop(['Timestamp','Rating'], axis=1)

In [13]:
print("Positive sample num: {}, Negative sample num: {}".format(pos_ratings_df.shape[0],neg_ratings_df.shape[0]))

Positive sample num: 194425, Negative sample num: 131124


In [14]:
# Parametes
batch_size_ = 64
eopch_num = 10000
learning_rate = 0.001

In [15]:
input_1_frames = 30
input_1_dimension = 2048
hidden_size = 16
input_2_dimension = 256
batch_size = 64
is_training = True
dropout = 0.5

In [16]:
from tensorflow.contrib import rnn 

In [32]:
tf.reset_default_graph()
with tf.name_scope('main_parameters'):
    x_lstm = tf.placeholder(tf.float32, [None, input_1_frames, input_1_dimension], name='Input_LSTM')
    x_lstm_seq_length = tf.placeholder(tf.int32, [None], name='Input_LSTM_Seq_lenght')
    x_lstm_keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    x_fusion = tf.placeholder(tf.float32, shape=[None, input_2_dimension], name='Input_fusion')
    
    y = tf.placeholder(tf.float32, shape=[None, 1], name='Output')

In [33]:
with tf.variable_scope('lstm_layer') as scope:
    seq_flat = tf.reshape(x_lstm, [-1, input_2_dimension])
    # [batch_size * max_length, hidden_size]
    seq_embed = tf.contrib.layers.fully_connected(seq_flat, hidden_size)
    seq_embed = tf.reshape(seq_embed, [-1, input_1_frames, hidden_size])
    
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=x_lstm_keep_prob)
    init_state = cell.zero_state(tf.shape(x_lstm)[0], dtype=tf.float32)

    outputs, states = tf.nn.dynamic_rnn(cell, x_lstm, sequence_length=x_lstm_seq_length,
                                        initial_state=init_state, dtype=tf.float32)
    
    
    batch_range = tf.range(tf.shape(outputs)[0])
    last_index = x_lstm_seq_length - 1
    indices = tf.stack([batch_range, last_index], axis=1)
    last = tf.gather_nd(outputs, indices)
    

In [34]:
with tf.variable_scope('fusion_layer') as scope:

        lstm_output_fc = tf.layers.dense(inputs=last, units=512, activation=tf.nn.relu)

        x_fusion_fc = tf.layers.dense(inputs=x_fusion, units=64, activation=tf.nn.relu)

        fused_tensor = tf.concat(axis=1,values=[lstm_output_fc, x_fusion_fc])

        last_layer = tf.layers.dense(inputs=fused_tensor, units=32, activation=tf.nn.relu) # ReLU

        y_pred = tf.layers.dense(inputs=last_layer, units=1, activation=tf.nn.softmax, name='regression_layer')

In [35]:
# x_lstm, x_fusion, y, y_pred

In [36]:
cost = tf.sqrt(tf.reduce_mean(tf.square(y - y_pred)))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [37]:
sess = tf.Session()


In [38]:
for batch in range(eopch_num):
    
    X_train_lstm, X_train_fusion, y_train = data_pipeline(training_df, recmodel.user_latent_traninig, recmodel.visual_features, batch_size_)   
    batch_lstm_xs = np.array(X_train_lstm[batch * batch_size_: (batch + 1) * batch_size_], dtype=float)
    batch_train_xs = np.array(X_train_fusion[batch * batch_size_: (batch + 1) * batch_size_], dtype=float)
    batch_ys = np.array(y_train[batch * batch_size_: (batch + 1) * batch_size_], dtype=float).reshape(batch_size_,1)
    
    print((batch_lstm_xs.dtype))
    print((batch_train_xs.dtype))
    print((batch_ys.dtype))
    
    break

float64
float64
float64


In [39]:
len(X_train_lstm)

64

In [40]:
type(batch_lstm_xs)

numpy.ndarray

In [41]:
batch_train_xs[0].shape

(256,)

In [43]:
np.array([2048]*batch_size).shape

(64,)

In [45]:
x_lstm_keep_prob

<tf.Tensor 'main_parameters/keep_prob:0' shape=<unknown> dtype=float32>

In [ ]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())


for batch in range(eopch_num):
        X_train_lstm, X_train_fusion, y_train = data_pipeline(training_df, recmodel.user_latent_traninig, recmodel.visual_features, batch_size_)  
        #print(len(y_train[batch * batch_size_: (batch + 1) * batch_size_]))
        batch_lstm_xs = np.array(X_train_lstm, dtype=float)
        batch_train_xs = np.array(X_train_fusion, dtype=float)
        batch_ys = np.array(y_train, dtype=float).reshape(batch_size_,1)
        batch_seq_lenth_xs = np.array([2048]*batch_size)
        
        #print("{} {} {} {}".format(batch_lstm_xs.shape, batch_train_xs.shape,batch_ys.shape,batch_seq_lenth_xs.shape))
        

        _, batch_loss = sess.run(
            [optimizer, cost],
            feed_dict={x_lstm: batch_lstm_xs,
                       x_lstm_seq_length:batch_seq_lenth_xs,
                       x_lstm_keep_prob:0.5,
                       x_fusion: batch_train_xs,
                       y: batch_ys})

        if batch%100==0:
            #msg = "Epoch {} - Training Batch Loss: {:.4f}"
            #print(msg.format((batch+1), batch_loss))
            
            X_train_lstm, X_train_fusion, y_train = data_pipeline(training_df, recmodel.user_latent_traninig, recmodel.visual_features, 1000)  
            #print(len(y_train[batch * batch_size_: (batch + 1) * batch_size_]))
            batch_lstm_xs = np.array(X_train_lstm, dtype=float)
            batch_train_xs = np.array(X_train_fusion, dtype=float)
            batch_ys = np.array(y_train, dtype=float).reshape(batch_size_,1)
            batch_seq_lenth_xs = np.array([2048]*batch_size)

            #print("{} {} {} {}".format(batch_lstm_xs.shape, batch_train_xs.shape,batch_ys.shape,batch_seq_lenth_xs.shape))


            predictions = sess.run(
                [y_pred],
                feed_dict={x_lstm: batch_lstm_xs,
                           x_lstm_seq_length:batch_seq_lenth_xs,
                           x_lstm_keep_prob:0.5,
                           x_fusion: batch_train_xs,
                           y: batch_ys})
            
            RMSE = (np.square(batch_ys - predictions)).mean()
            
            #msg = "Epoch {} - Training Batch Loss: {:.4f}"
            #print(msg.format((batch+1), batch_loss))
            
            msg = "Epoch {} - RMSE: {}"
            print(msg.format((batch+1), RMSE))


Epoch 1 - RMSE: 1.6875
Epoch 101 - RMSE: 1.75
Epoch 201 - RMSE: 1.6875


In [26]:
import time.time as time

ModuleNotFoundError: No module named 'time.time'; 'time' is not a package

In [28]:
with tf.variable_scope('lstm_layer') as scope:
    state_per_layer_list = tf.unstack(x_lstm, axis=0)
    rnn_tuple_state = tuple([tf.contrib.rnn.LSTMStateTuple(state_per_layer_list[idx][0], state_per_layer_list[idx][1]) for idx in range(num_layers)])
    # create an LSTM cell to be unrolled
    cell = tf.contrib.rnn.LSTMCell(hidden_size, forget_bias=1.0)
    
    # add a dropout wrapper if training
    cell = tf.contrib.rnn.MultiRNNCell([cell for _ in range(num_layers)], state_is_tuple=True)

    # cell = tf.contrib.rnn.LSTMCell(state_size, state_is_tuple=True)
    # cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers, state_is_tuple=True)

    output, state = tf.nn.dynamic_rnn(cell, x_lstm, dtype=tf.float32, initial_state=rnn_tuple_state)
    # reshape to (batch_size * num_steps, hidden_size)
    lstm_output = tf.reshape(output, [-1, hidden_size])

    lstm_output = lstm_output/tf.reduce_max(tf.abs(lstm_output))

ValueError: Cannot infer num from shape (?, 30, 2048)

In [16]:
rnn_tuple_state[0]

LSTMStateTuple(c=<tf.Tensor 'lstm_layer/strided_slice:0' shape=(2048,) dtype=float32>, h=<tf.Tensor 'lstm_layer/strided_slice_1:0' shape=(2048,) dtype=float32>)

In [17]:
rnn_tuple_state[1]

LSTMStateTuple(c=<tf.Tensor 'lstm_layer/strided_slice_2:0' shape=(2048,) dtype=float32>, h=<tf.Tensor 'lstm_layer/strided_slice_3:0' shape=(2048,) dtype=float32>)

In [ ]:
# Recurrent Neural Network
with tf.name_scope("rnn"):
    cell = self._get_cell(hidden_size, cell_type)
    cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=self.dropout_keep_prob)
    all_outputs, _ = tf.nn.dynamic_rnn(cell=cell,
                                       inputs=self.embedded_chars,
                                       sequence_length=text_length,
                                       dtype=tf.float32)
    self.h_outputs = self.last_relevant(all_outputs, text_length)

    
    
    